In [6]:
cd /projects/crunchie/boyanova/EEG_Things/eeg_prep/

/projects/crunchie/boyanova/EEG_Things/eeg_prep


In [7]:
import pandas as pd
import numpy as np
import random
import sys 
import os

In [8]:
def create_changes(list_length, prec):
    # Number of instances to set as True (2%)
    num_true = int(list_length * prec)

    # Create lists of False values
    catch = [False] * list_length

    # Randomly select indices for fixing and imaging channels
    catch_indices = random.sample(range(list_length), num_true)

    for idx in catch_indices:
        catch[idx] = True
    
    return np.array(catch)

In [9]:
#--- load data and select seqs
experiment_path = ".\\stimuli\\"

exp_imgs = pd.read_csv("exp_stimuli.csv")
print("Experimental stimuli in!")
del exp_imgs['Unnamed: 0']

subj = 0
list_len = 60
prec = 0.05

det_seqs = [[0,1,2,3],
            [0,1,3,2],
            [0,2,1,3],
            [0,2,3,1],
            [0,3,1,2],
            [0,3,2,1]]


bl_cond = ["random", "determ"]
task_cond = ["att_img", "att_fix"]


main = exp_imgs[exp_imgs.cond == "main"]
alt = exp_imgs[exp_imgs.cond == "alt"]

main_imgs = main.image_name.values
main_IDs = main.image_ID.values
alt_imgs = alt.image_name.values
alt_IDs = alt.image_ID.values

for seq_id, seq in enumerate(det_seqs):
    for bl_type_id, bl_type in enumerate(bl_cond):
        
        for task_id, task in enumerate(task_cond):
            
            
            catch_trials = create_changes(list_len * len(seq), prec)
            repeated_array = [string for _ in range(list_len) for string in main_imgs[seq]]
            repeated_ID = [string for _ in range(list_len) for string in main_IDs[seq]]
            bl_seq = np.array(repeated_array)
            bl_seqID  = np.array(repeated_ID)
            image_type = ["main" for _ in range(len(catch_trials))]
            
            if bl_type == "random":
                ## shuffle
                np.random.seed(seed=seq_id)
                np.random.shuffle(repeated_array)
                np.random.seed(seed=seq_id)
                np.random.shuffle(repeated_ID)

                bl_seq = np.array(repeated_array)
                bl_seqID = np.array(repeated_ID)
                
                trigN = 11
            if bl_type == "determ":
                trigN = 1
                
            fix_cross_col = np.array(["black" for _ in range(len(catch_trials))])
            if task == "att_fix":
                fix_cross_col[np.where(catch_trials == True)] = "red"
                for fix_ch in np.arange(0, list_len * len(alt_imgs))[catch_trials]:
                    category = main[main.image_name == bl_seq[fix_ch]].image_category.values
                    bl_seqID[fix_ch] = alt[alt.image_category == category[0]].image_ID.values[0]
                    
                
            else:
                for im_ch in np.arange(0, list_len * len(alt_imgs))[catch_trials]:
                    category = main[main.image_name == bl_seq[im_ch]].image_category.values
                    bl_seq[im_ch] = alt[alt.image_category == category[0]].image_name.values[0]
                    bl_seqID[im_ch] = alt[alt.image_category == category[0]].image_ID.values[0]
                    image_type[im_ch] = "alt"
                
            trial_file = {"image_name": list(bl_seq),
                         "image_ID": list(bl_seqID),
                         "image_type": list(image_type),
                          "image_path": [experiment_path + im for im in bl_seq],
                          "catch_trial": catch_trials,
                          "corr_ans": ["space" if value else "shit" for value in catch_trials],
                          "fix_cross_col": list(fix_cross_col),
                          "block_type": [bl_type for _ in range(len(bl_seq))],
                          "trig": [str(tr + trigN) for tr in bl_seqID]
                         }
            
            trial_file = pd.DataFrame(trial_file)
            trial_file.to_csv("./blocks/{}_{}_{:02d}.csv".format(bl_type, task, seq_id), header = True)
            print("{}_{}_{:02d}.csv Saved!".format(bl_type, task, seq_id))
            print("-" * 20)

            


Experimental stimuli in!
random_att_img_00.csv Saved!
--------------------
random_att_fix_00.csv Saved!
--------------------
determ_att_img_00.csv Saved!
--------------------
determ_att_fix_00.csv Saved!
--------------------
random_att_img_01.csv Saved!
--------------------
random_att_fix_01.csv Saved!
--------------------
determ_att_img_01.csv Saved!
--------------------
determ_att_fix_01.csv Saved!
--------------------
random_att_img_02.csv Saved!
--------------------
random_att_fix_02.csv Saved!
--------------------
determ_att_img_02.csv Saved!
--------------------
determ_att_fix_02.csv Saved!
--------------------
random_att_img_03.csv Saved!
--------------------
random_att_fix_03.csv Saved!
--------------------
determ_att_img_03.csv Saved!
--------------------
determ_att_fix_03.csv Saved!
--------------------
random_att_img_04.csv Saved!
--------------------
random_att_fix_04.csv Saved!
--------------------
determ_att_img_04.csv Saved!
--------------------
determ_att_fix_04.csv Sav

In [23]:
experiment_path = ".\\blocks\\"
instructions = ["Pay attention to the images!", "Pay attention to the fixation cross!"]

block_names = os.listdir("/projects/crunchie/boyanova/EEG_Things/eeg_prep/blocks")
image_random = [bl for bl in block_names if bl.split("_")[0] == "random" and bl.split("_")[-2] == "img"]
image_det = [bl for bl in block_names if bl.split("_")[0] == "determ" and bl.split("_")[-2] == "img"]
fix_random = [bl for bl in block_names if bl.split("_")[0] == "random" and bl.split("_")[-2] == "fix"]
fix_det = [bl for bl in block_names if bl.split("_")[0] == "determ" and bl.split("_")[-2] == "fix"]

for subject_number in range(20):

    # Initialize the block orders
    block_order_1 = []
    block_order_2 = []

    # Determine the order based on subject number
    if subject_number % 4 == 0:
        int1 = 0
        int2 = 1

        for i in range(6):
            block_order_1.append(image_det[i])
            block_order_1.append(image_random[i])
            block_order_2.append(fix_random[i])
            block_order_2.append(fix_det[i])

    if subject_number % 4 == 1:
        int1 = 0
        int2 = 1

        for i in range(6):
            block_order_1.append(image_random[i])
            block_order_1.append(image_det[i])
            block_order_2.append(fix_det[i])
            block_order_2.append(fix_random[i])

    if subject_number % 4 == 2:
        int1 = 1
        int2 = 0

        for i in range(6):
            block_order_1.append(fix_det[i])
            block_order_1.append(fix_random[i])
            block_order_2.append(image_random[i])
            block_order_2.append(image_det[i])
    if subject_number % 4 == 3:
        int1 = 1
        int2 = 0

        for i in range(6):
            block_order_1.append(fix_random[i])
            block_order_1.append(fix_det[i])
            block_order_2.append(image_det[i])
            block_order_2.append(image_random[i])




    blocks_1 = pd.DataFrame({"block_name":block_order_1,
              "block_type": [bl.split("_")[0] for bl in block_order_1],
              "block_path": [experiment_path + bl for bl in block_order_1],
             "block_task": [instructions[int1] for _ in range(len(block_order_1))]})

    blocks_2 = pd.DataFrame({"block_name":block_order_2,
                  "block_type": [bl.split("_")[0]  for bl in block_order_2],
                  "block_path": [experiment_path + bl for bl in block_order_2],
                 "block_task": [instructions[int2] for _ in range(len(block_order_2))]})

    blocks_1.to_csv("/projects/crunchie/boyanova/EEG_Things/eeg_prep/sessions/session01_subj{:02d}.csv".format(subject_number))
    blocks_2.to_csv("/projects/crunchie/boyanova/EEG_Things/eeg_prep/sessions/session02_subj{:02d}.csv".format(subject_number))

    print("Sessions for subject {:02d} created!".format(subject_number))
    print("-" * 20)

Sessions for subject 00 created!
--------------------
Sessions for subject 01 created!
--------------------
Sessions for subject 02 created!
--------------------
Sessions for subject 03 created!
--------------------
Sessions for subject 04 created!
--------------------
Sessions for subject 05 created!
--------------------
Sessions for subject 06 created!
--------------------
Sessions for subject 07 created!
--------------------
Sessions for subject 08 created!
--------------------
Sessions for subject 09 created!
--------------------
Sessions for subject 10 created!
--------------------
Sessions for subject 11 created!
--------------------
Sessions for subject 12 created!
--------------------
Sessions for subject 13 created!
--------------------
Sessions for subject 14 created!
--------------------
Sessions for subject 15 created!
--------------------
Sessions for subject 16 created!
--------------------
Sessions for subject 17 created!
--------------------
Sessions for subject 18 crea